# Tutorial 8: Exploratory Data Analysis

## Background

As a data scientist at Heilbronn Hospital, you've been tasked with conducting a thorough exploratory data analysis (EDA) of patient records. The medical team needs to understand patterns in patient care, identify data quality issues, and gain insights that could lead to improved treatment protocols. Your analysis will help the hospital make data-driven decisions about resource allocation and patient care strategies.

1. Data Quality Assessment: The ICU's electronic health records system occasionally has gaps in data collection due to emergency situations or equipment issues. Import the dataset 'icu_data_example.csv' into a DataFrame, then identify which vital signs and patient information are most frequently missing by calculating the percentage of missing values in each column. This analysis will help the hospital improve their data collection protocols.


In [ ]:
import pandas as pd

df = pd.read_csv('../icu_data_example.csv')

missing_percentages = df.isnull().mean() * 100

missing_percentages_sorted = missing_percentages.sort_values(ascending=False)

print(missing_percentages_sorted)


2. Data Imputation Strategy: For accurate patient assessment, we need to handle missing data carefully. Implement a for-loop to iterate over all vital sign measurements and replace missing values with the median of the respective column, but only where the missing data is below 20%. This conservative approach ensures we only fill in gaps where we have sufficient reliable data to make educated estimates.


In [ ]:
import pandas as pd

df = pd.read_csv('../icu_data_example.csv')
vital_sign_cols = df.select_dtypes(include=['float64', 'int64']).columns

#prettifying the output
GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"

for col in vital_sign_cols:
    missing_pct = df[col].isnull().mean() * 100
    if missing_pct < 20:
        median_value = df[col].median()
        df.fillna({col: median_value}, inplace=True)
        print(f"{GREEN}Imputed '{col}' with median ({median_value}) - missingness {missing_pct:.2f}%{RESET}")
    else:
        print(f"{RED}Skipped '{col}' - missingness {missing_pct:.2f}%{RESET}")

3. Outlier Detection: Medical equipment sometimes produces erroneous readings, and transcription errors can occur. Use the Interquartile Range (IQR) method to identify potentially abnormal values in patients' 'weight' and 'Heart Rate' measurements. This will help the medical team verify these readings and ensure accurate patient monitoring.


In [ ]:
import pandas as pd

df = pd.read_csv('../icu_data_example.csv')

checkcols = ["weight", "Heart Rate"]

pos_outlier = {}

for col in checkcols:
    col_data = df[col].dropna()
    Q1 = col_data.quantile(0.25)
    Q3 = col_data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    pos_outlier[col] = outliers.index.tolist()
    print(f"\nColumn: {col}")
    print(f"Q1: {Q1}")
    print(f"Q3: {Q3}")
    print(f"IQR: {IQR}")
    print(f"Lower bound: {lower_bound}")
    print(f"Upper bound: {upper_bound}")
    print(f"Outliers: {outliers.shape[0]}")

4. Age Group Analysis: To better understand the demographics of ICU admissions, create a new column that categorizes patients into clinically relevant age groups (0-20, 21-40, 41-60, etc.). This categorization will help the hospital plan age-specific care protocols and resource allocation.


In [ ]:
import pandas as pd

df = pd.read_csv('../icu_data_example.csv')

bins = [0, 20, 40, 60, 80, 100]
labels = ['0-20', '21-40', '41-60', '61-80', '81-100']

df['AgeGroup'] = pd.cut(df['age'], bins=bins, labels=labels, right=True, include_lowest=True)

print(df['AgeGroup'].value_counts().sort_index())

5. Gender-based Analysis: The medical team wants to investigate if there are gender-specific patterns in ICU care. Group the data by gender and calculate the median length of ICU stay for each group. This analysis could reveal if certain gender-specific health factors influence recovery time.

In [ ]:
import pandas as pd

df = pd.read_csv('../icu_data_example.csv')

st_by_gender = df.groupby('gender')['los'].median()
print("Median ICU Stay by Gender:")
print(st_by_gender)

6. Length of Stay Classification: For better resource planning, create a new column that categorizes ICU stays into three groups: Short (<5 days), Medium (5-10 days), and Long (>10 days). Count the number of patients in each category. This information will help the hospital optimize bed allocation and staffing levels.


In [ ]:
import pandas as pd

df = pd.read_csv('../icu_data_example.csv')

def classify_stay(days):
    if pd.isnull(days):
        return 'Unknown'
    elif days < 5:
        return 'Short'
    elif 5 <= days <= 10:
        return 'Medium'
    else:
        return 'Long'

df['StayLengthCategory'] = df['los'].apply(classify_stay)
stay_counts = df['StayLengthCategory'].value_counts()
print("Patients per stay length category:")
print(stay_counts)

7. Data Integration: The hospital maintains additional patient information in a separate file 'icu_data_additional.csv'. Merge this dataset with our main ICU data using a common patient identifier. This will provide a more comprehensive view of each patient's medical history and current condition.


In [ ]:
import pandas as pd

df = pd.read_csv('../icu_data_example.csv')
additional_df = pd.read_csv('../icu_data_example.csv')

merged_df = pd.merge(df, additional_df, how='left', on='stay_id')

print("merged df shape:", merged_df.shape)
print("columns in merged df:")
print(merged_df.columns)

8. Distribution Analysis: Create histograms of patient ages and heart rates using pandas or matplotlib. Understanding the distribution of these vital statistics will help medical staff better prepare for the typical patient profile and identify unusual cases that might need special attention.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('../icu_data_example.csv')

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(df['age'].dropna(), bins=20, color='skyblue', edgecolor='black')
plt.xlabel('age')
plt.ylabel('Patients number')
plt.title('Distribution of patient ages')
plt.subplot(1, 2, 2)
plt.hist(df['Heart Rate'].dropna(), bins=20, color='salmon', edgecolor='black')
plt.xlabel('Heart Rate')
plt.ylabel('Number of patients')
plt.title('Heart rates distribution')
plt.tight_layout()
plt.show()